In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, shape
import geopandas as gpd
import pickle

# Import custom functions from `scripts` folder
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scripts.clean_tweets import geometrize_tweets, convert_shapefile_crs, find_frequencies
from scripts.home_location import assign_home_location

np.random.seed(42)
pd.set_option('display.max_columns', 999)

# Loading data

In [ ]:
ba13 = pd.read_csv('../data/tweets/ba_2013.csv')

/scratch/public/jsp/twitter-and-displacement/.env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
ba_shapefiles = gpd.read_file('../data/shapefiles/buenos_aires_shapefiles/BA_2010.shp')
ba_shapefiles = convert_shapefile_crs(ba_shapefiles)

# Filtering data and adding tracts

In [ ]:
# Compute summary stats
num_tweets_13, num_users_13, median_tweets_13 = len(ba13), ba13['u_id'].nunique(), ba13.groupby('u_id').size().median()
percentile_99 = ba13.groupby('u_id').size().quantile(.99)

In [ ]:
# Filter based on median tweets for 2013
ba13_filtered = ba13.groupby('u_id').filter(lambda group: (len(group) >= median_tweets_13) & (len(group) <= percentile_99) )

In [ ]:
# Geometrize tweets based on lat/lon
ba13_filtered = geometrize_tweets(ba13_filtered)

In [ ]:
# Add datetime
ba13_filtered['timestamp'] = pd.to_datetime(ba13_filtered['created_at'] // 1000, unit='s')
ba13_filtered['date'] = ba13_filtered['timestamp'].dt.date
ba13_filtered['hour'] = ba13_filtered['timestamp'].dt.hour

In [ ]:
# Spatial join for tracts; add home locations
ba13_filtered = gpd.sjoin(ba13_filtered, ba_shapefiles, how='left', op='intersects')
ba13_filtered['home'] = assign_home_location(ba13_filtered, tract='codigo')

In [6]:
# Save data for easy loading in future
# with open('../data/ba13_home.pkl', 'wb') as file:
#     pickle.dump(ba13_filtered, file, protocol=4)
with open('../data/ba13_home.pkl', 'rb') as file:
    ba13_filtered = pickle.load(file)
        
ba13_filtered['is_home'] = ba13_filtered['home'] == ba13_filtered['codigo']

In [ ]:
# How many of the filtered tweets have a home assignment?
print((~ba13_filtered['home'].isnull()).mean())

In [ ]:
# Ratio of users that have home location
ba13_filtered.groupby(['u_id']).first()['is_home'].mean()

In [ ]:
# Ratio of filtered 2013 tweets in which tweet was made from home tract 
ba13_filtered['is_home'].mean()

In [ ]:
### Plot 1% of 2013 tweets
# Blue tweets: Resident
# Red tweets: Non-resident
# Gray (barely visible beneath the blue): Buenos Aires shapefile
fig, ax = plt.subplots(figsize=(10, 7))
ba_shapefiles['geometry'].plot(ax=ax, color='gray')
smpl = ba13_filtered.sample(frac=0.01, random_state=42)
smpl[~smpl['is_home']].plot(ax=ax, marker='o', color='red', alpha=0.01, label='Red: Non-Resident')
smpl[smpl['is_home']].plot(ax=ax, marker='o', color='blue', alpha=0.01, label='Blue: Resident')
plt.legend()
plt.axis([-58.6, -58.3, -34.75, -34.5])
plt.title('Buenos Aires tweets, 2013');

# Filtered

In [11]:
# Save data for easy loading in future
# with open('../data/ba13_home.pkl', 'wb') as file:
#     pickle.dump(ba13_filtered, file, protocol=4)
with open('../data/ba13_home.pkl', 'rb') as file:
    ba13_filtered = pickle.load(file)

ba13_filtered['home'] = assign_home_location(ba13_filtered, tract='codigo')
ba13_filtered['is_home'] = ba13_filtered['home'] == ba13_filtered['codigo']

In [ ]:
# How many of the filtered tweets have a home assignment?
print((~ba13_filtered.loc[~ba13_filtered['codigo'].isnull(), 'home'].isnull()).mean())

In [ ]:
# Ratio of users that have home location
ba13_filtered[~ba13_filtered['codigo'].isnull()].groupby(['u_id']).first()['is_home'].mean()

In [ ]:
# Ratio of filtered 2013 tweets in which tweet was made from home tract 
ba13_filtered.loc[~ba13_filtered['codigo'].isnull(), 'is_home'].mean()

In [ ]:
### Plot 1% of 2013 tweets
# Blue tweets: Resident
# Red tweets: Non-resident
# Gray (barely visible beneath the blue): Buenos Aires shapefile
fig, ax = plt.subplots(figsize=(10, 7))
ba_shapefiles['geometry'].plot(ax=ax, color='gray')
smpl = ba13_filtered[~ba13_filtered['codigo'].isnull()].sample(frac=0.01, random_state=42)
smpl[~smpl['is_home']].plot(ax=ax, marker='o', color='red', alpha=0.01, label='Red: Non-Resident')
smpl[smpl['is_home']].plot(ax=ax, marker='o', color='blue', alpha=0.01, label='Blue: Resident')
plt.legend()
plt.axis([-58.6, -58.3, -34.75, -34.5])
plt.title('Buenos Aires tweets, 2013');

In [19]:
(
    ba13_filtered
    .loc[:, ['codigo', 'is_home']]
    .groupby('codigo')
    .agg([len, np.mean])
    .rename(columns={'len':'count', 'mean':'home_tweet_ratio'})
).to_csv('../data/ba13_home_by_codigo.csv')